In [ ]:
!pip install transformers langchain torch gradio

In [ ]:
# Install the classic package
!pip install langchain_classic

In [ ]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
from langchain_classic.memory import ConversationBufferWindowMemory
import gradio as gr

In [ ]:
model_name="meta-llama/Llama-2-7b-chat-hf"
token="hf_GXBSmMplmesJ####FwzeuEcD#"

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

using device: cuda


In [ ]:
model=AutoModelForCausalLM.from_pretrained(model_name, token=token).to(device)
tokenizer=AutoTokenizer.from_pretrained(model_name, token=token)

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def chatbot_responce(user_input):

  inputs=tokenizer(user_input,return_tensors="pt",max_length=512,truncation=True).to(device)


  outputs=model.generate(**inputs)


  bot_reply=tokenizer.decode(outputs[0],skip_special_tokens=True)


  if bot_reply.startswith(user_input):
      bot_reply=bot_reply[len(user_input):].strip()
  return bot_reply

In [ ]:
def chatbot_interface(user_input,history):

  response=chatbot_responce(user_input)

  history.append((user_input,response))

  return history,""


demo=gr.Blocks()

In [ ]:
with demo:
  gr.Markdown("## GENAI Chatbot")
  chatbot=gr.Chatbot()
  with gr.Row():
    user_input=gr.Textbox(placeholder="Type your message here",lines=1)
    send_button=gr.Button("Send")

  history=gr.State([])

  send_button.click(chatbot_interface,inputs=[user_input,history],outputs=[chatbot, user_input])
  user_input.submit(chatbot_interface,inputs=[user_input,history],outputs=[chatbot, user_input])

/tmp/ipython-input-1170503289.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot()
/tmp/ipython-input-1170503289.py:3: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot=gr.Chatbot()


In [ ]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b47b14568e9f942464.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from huggingface_hub import login

# Ensure the token variable is available (it should be from previous cells)
if 'token' in locals() or 'token' in globals():
    login(token=token)
    print("Successfully logged in to Hugging Face.")
else:
    print("Hugging Face token not found. Please ensure it's defined.")

Successfully logged in to Hugging Face.


In [ ]:
from huggingface_hub import HfApi

try:
    api = HfApi(token=token)
    user_info = api.whoami()
    print(f"Successfully authenticated as: {user_info['name']}")
    # Further check for model access if needed, e.g., by trying to fetch model info
    model_info = api.model_info(model_name)
    print(f"Successfully retrieved info for model: {model_info.modelId}")
    print("Your Hugging Face token is valid and has access to the specified model.")
except Exception as e:
    print(f"Error during Hugging Face token verification or model access check: {e}")
    print("Please ensure your token is correct, has the necessary permissions, and you have accepted the model's terms on Hugging Face.")

Successfully authenticated as: SurajRavirala
Successfully retrieved info for model: meta-llama/Llama-2-7b-chat-hf
Your Hugging Face token is valid and has access to the specified model.
